In [72]:
import requests
from bs4 import BeautifulSoup
import re
from collections import Counter
import matplotlib.pyplot as plt #畫圖用
global static #增加一個全域變數用來紀錄統計結果
static={}

In [73]:
with open('./wordlen.txt') as f: #打開我們要比對的字典檔
    wordlen = f.read().split('\n')
del wordlen[0]
wordlen[:10] 

['C', 'C++', 'C#', 'PYTHON', 'JAVA', 'JAVASCRIPT', 'PHP', 'HTML', 'SQL', 'CSS']

In [74]:
def inner_word(url): #一個用來分析內頁的function
    
    inner = requests.get(url)  #用get連到網頁
    innersoup = BeautifulSoup(inner.text, 'lxml')  #放到BeautifulSoup解析
    innerselect2 = str(innersoup.select('div.job-detail-box > dl')).upper()  
    #找到可能會出現找尋目標的區塊將全部英文變大寫
    innerselect = str(innersoup.select('div > p')).upper()
    #同上
    lan1=re.findall('[A-Z]+[+#?]*' , innerselect)
    #用正規法抓出每個英文單字
    lan2=re.findall('[A-Z]+[+#?]*' , innerselect2)
    #同上 第二區塊的
    lan1.extend(lan2)
    #將兩個list合併
    if 'HTML#' in lan1:
        lan1.remove('HTML#')
    #排除調內文出現的雜訊
    if 'HTML?' in lan1:
        lan1.remove('HTML?')
    #雜訊2
    lan1 = list(set(lan1))
    #清除調list內重複的部份
    
    for i in lan1: #將每個元素都跑過 內容會顯示在i
        if i in wordlen: #如果list裡面的字有出現在我們的字典檔中
            if i in static:  #如果static已經有這個字 就在value+1
                static[i] += 1
            else:
                static[i] = 1 #如果static統計的字典檔還沒有這個字 就新增
    return static


In [75]:
def page(url):  #這是用來抓出搜尋畫面中所有的case連結
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    links = soup.select('li.title > a')
    for link in links:
        inner_word(str(link['href']))#把取得的連結丟到上面inner_word function去解析
        print("%s" %  link['href'])
        print("=" * 50)
    return static
    

In [76]:
for i in range(1,1+1): #換頁 爬的頁數
    print(('Page'+str(i)+' ')*50) #顯示出現在爬到那一頁 讓我知道進度如何而已
    page('https://www.518.com.tw/job-index-P-'+str(i)+'.html?i=1&am=1&ab=2032001,2032002,') 
    #把換頁後的url丟到上面的page function

Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 Page1 
https://www.518.com.tw/MIS網管(駐點香港)-中國-香港區-job-1349006.html?kw=&pi=1
https://www.518.com.tw/V-最熱門-超簡易的 手工撕膜 也能高時薪206月領37K  近頂埔捷運-新北市-板橋區-job-1338811.html?kw=&pi=1
https://www.518.com.tw/IT及MIS工程師-台北市-大同區-job-1353473.html?kw=&pi=1
https://www.518.com.tw/營運管理師 系統整合 ERP專案師-台北市-中山區-job-1352784.html?kw=&pi=1
https://www.518.com.tw/PHP網頁程式工程師-台南市-東區-job-154420.html?kw=&pi=1
https://www.518.com.tw/資訊助理人員 (實習生亦可)-新北市-林口區-job-1223414.html?kw=&pi=1
https://www.518.com.tw/徵美工助理兼網拍內勤人員-台中市-大里區-job-208730.html?kw=&pi=1
https://www.518.com.tw/網頁設計師-高雄市-仁武區-job-1293901.html?kw=&pi=1
https://www.518.com.tw/駐點資訊工程師  工作地點高雄-高雄市-前鎮區-job-1332707.html?kw=&pi=1
https://www.518.com.tw/Internet程式設計-早班-嘉義市-東區-job-12

In [77]:
static
#結果

{'C': 14,
 'C#': 1,
 'C++': 4,
 'CSS': 8,
 'GO': 1,
 'HTML': 12,
 'JAVA': 5,
 'JAVASCRIPT': 3,
 'PHP': 8,
 'PYTHON': 1,
 'SQL': 5}

In [78]:
#測試
inner_word("https://www.518.com.tw/unity%E5%B7%A5%E7%A8%8B%E5%B8%AB-%E9%AB%98%E9%9B%84%E5%B8%82-%E5%89%8D%E9%8E%AE%E5%8D%80-job-1344470.html?kw=C&pi=1")

{'C': 14,
 'C#': 2,
 'C++': 5,
 'CSS': 8,
 'GO': 1,
 'HTML': 12,
 'JAVA': 6,
 'JAVASCRIPT': 3,
 'PHP': 8,
 'PYTHON': 1,
 'SQL': 5}

In [79]:
# def display_top_features(weights, names, top_n, select=abs):
#     top_features = sorted(zip(weights, names), key=lambda x: select(x[0]), reverse=True)[:top_n]
#     top_weights = [x[0] for x in top_features]
#     top_names = [x[1] for x in top_features]

#     fig, ax = plt.subplots(figsize=(10,8))
#     ind = np.arange(top_n)
#     bars = ax.bar(ind, top_weights, color='blue', edgecolor='black')
#     for bar, w in zip(bars, top_weights):
#         if w < 0:
#             bar.set_facecolor('red')

#     width = 0.30
#     ax.set_xticks(ind + width)
#     ax.set_xticklabels(top_names, rotation=45, fontsize=8, fontdict={'fontname': 'LiHei Pro', 'fontsize':12})

#     plt.show(fig)

In [88]:
import  matplotlib.pyplot as plt
import numpy as np
for i, key in enumerate(static):#Circulate both index and value(Here is key)
    plt.bar(i, static[key], color='r', width=0.2)
plt.xticks(np.arange(len(static))+0.1, static.keys())#Translation
plt.yticks(static.values())
plt.grid(True)

plt.show()

TypeError: len() of unsized object

In [86]:
static.keys()

dict_keys(['C', 'HTML', 'CSS', 'JAVA', 'PHP', 'C++', 'JAVASCRIPT', 'PYTHON', 'SQL', 'GO', 'C#'])

In [87]:
type (static.values())

dict_values

In [90]:
len(static)

11

In [91]:
import  matplotlib.pyplot as plt
import numpy as np
dict = {'A': 40, 'B': 70, 'C': 30, 'D': 85}
for i, key in enumerate(dict):#Circulate both index and value(Here is key)
    plt.bar(i, dict[key], color='r', width=0.2)
plt.xticks(np.arange(len(dict))+0.1, dict.keys())#Translation
plt.yticks(dict.values())
plt.grid(True)
plt.show()

TypeError: len() of unsized object